In [8]:
data_dir='/home/ubuntu/intelliscope/ecg_data/'# fill # data_dir/ecg_other_train, ecg_other_val, ... ecg_other_train_imb, ..should be there
model_dir='/home/ubuntu/intelliscope/notebooks/physionet2017/ecg/models/'

In [9]:
import os
a1=len(os.listdir('/home/ubuntu/intelliscope/ecg_data/ecg_other_train_imb/0'))
a2=len(os.listdir('/home/ubuntu/intelliscope/ecg_data/ecg_other_val_imb/0'))
a3=len(os.listdir('/home/ubuntu/intelliscope/ecg_data/ecg_other_test_imb/0'))

a4=len(os.listdir('/home/ubuntu/intelliscope/ecg_data/ecg_other_train_imb/1'))
a5=len(os.listdir('/home/ubuntu/intelliscope/ecg_data/ecg_other_val_imb/1'))
a6=len(os.listdir('/home/ubuntu/intelliscope/ecg_data/ecg_other_test_imb/1'))

a7=len(os.listdir('/home/ubuntu/intelliscope/ecg_data/ecg_other_train/0'))
a8=len(os.listdir('/home/ubuntu/intelliscope/ecg_data/ecg_other_val/0'))
a9=len(os.listdir('/home/ubuntu/intelliscope/ecg_data/ecg_other_test/0'))

a10=len(os.listdir('/home/ubuntu/intelliscope/ecg_data/ecg_other_train/1'))
a11=len(os.listdir('/home/ubuntu/intelliscope/ecg_data/ecg_other_val/1'))
a12=len(os.listdir('/home/ubuntu/intelliscope/ecg_data/ecg_other_test/1'))

print(f'imb:: N:{a1+a2+a3}, A:{a4+a5+a6}')
print(f'balanceed:: N:{a7+a8+a9}, A:{a10+a11+a12}')

imb:: N:5068, A:757
balanceed:: N:757, A:757


In [10]:
PARAMS = {
          'epochs' : 15,
         'validation_split':0.15,
         'batch_size' : 32,
            'model': 'simple',
          'data_balanced':True,
            'num_epochs':15,
            'opt':'adam',
            'tags': ['instance_norm added', 'dropout added'],
         }

In [11]:
import os
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda,BatchNormalization
from tensorflow.keras import backend as K
from tensorflow_addons.layers import InstanceNormalization

from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda


from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import tensorflow as tf

In [12]:


#k1,k2=100,100
k1,k2=272, 462

model_simple1=Sequential([
    #tf.keras.layers.experimental.preprocessing.Resizing(k1, k2, interpolation="bilinear")
    Lambda(lambda image: tf.image.resize( image, (64, 64), method = tf.image.ResizeMethod.BICUBIC, preserve_aspect_ratio = True),input_shape=(k1, k2,3)),
    Conv2D(16, (5, 5),activation='relu'),
    Conv2D(32, (5, 5),activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3),activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(3840, activation='relu'),
    Dense(64,activation='relu'),
    Dropout(0.5),
    Dense(2,activation='softmax')])

model_simple2=Sequential([
    #tf.keras.layers.experimental.preprocessing.Resizing(k1, k2, interpolation="bilinear")
    Lambda(lambda image: tf.image.resize( image, (64, 64), method = tf.image.ResizeMethod.BICUBIC, preserve_aspect_ratio = True),input_shape=(k1, k2,3)),
    Conv2D(16, (5, 5),activation='relu'),
    InstanceNormalization(axis=3, center=True, scale=True,beta_initializer="random_uniform",gamma_initializer="random_uniform"),
    Dropout(0.2),
    Conv2D(32, (5, 5),activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    InstanceNormalization(axis=3, center=True, scale=True,beta_initializer="random_uniform",gamma_initializer="random_uniform"),
    Dropout(0.2),
    Conv2D(64, (3, 3),activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    InstanceNormalization(axis=3, center=True, scale=True,beta_initializer="random_uniform",gamma_initializer="random_uniform"),
    Dropout(0.2),
    Flatten(),
    Dense(3840, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(64,activation='relu'),
    Dropout(0.5),
    Dense(2,activation='softmax')])


model=model_simple2
model.compile(loss=keras.losses.CategoricalCrossentropy(),
              optimizer=PARAMS['opt'],
              metrics=[keras.metrics.CategoricalAccuracy(), keras.metrics.Recall(),keras.metrics.Precision()])

PARAMS['model']=model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_5 (Lambda)            (None, 38, 64, 3)         0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 34, 60, 16)        1216      
_________________________________________________________________
instance_normalization_6 (In (None, 34, 60, 16)        32        
_________________________________________________________________
dropout_13 (Dropout)         (None, 34, 60, 16)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 30, 56, 32)        12832     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 15, 28, 32)        0         
_________________________________________________________________
instance_normalization_7 (In (None, 15, 28, 32)       

In [13]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

batch_size=PARAMS['batch_size']

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        data_dir+'ecg_other_train/',  # this is the target directory
        target_size=(k1, k2),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

validation_generator = val_datagen.flow_from_directory(
        data_dir+'ecg_other_val/',
        target_size=(k1, k2),
        batch_size=batch_size,
        class_mode='categorical')

history=model.fit_generator(
        train_generator,
        #steps_per_epoch=2000 // batch_size,
        epochs=PARAMS['num_epochs'],
        validation_data=validation_generator,
        #validation_steps=800 // batch_size
)



Using TensorFlow backend.


Found 1004 images belonging to 2 classes.
Found 254 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 32 steps, validate for 8 steps
Epoch 1/15
32/32 [==============================] - 41s 1s/step - loss: 1.0072 - categorical_accuracy: 0.5189 - recall_2: 0.5189 - precision_2: 0.5189 - val_loss: 0.6931 - val_categorical_accuracy: 0.5000 - val_recall_2: 0.5000 - val_precision_2: 0.5000
Epoch 2/15
32/32 [==============================] - 38s 1s/step - loss: 0.8215 - categorical_accuracy: 0.5478 - recall_2: 0.5478 - precision_2: 0.5478 - val_loss: 0.6931 - val_categorical_accuracy: 0.5000 - val_recall_2: 0.5000 - val_precision_2: 0.5000
Epoch 3/15
32/32 [==============================] - 38s 1s/step - loss: 0.7029 - categorical_accuracy: 0.5697 - recall_2: 0.5697 - precision_2: 0.5697 - val_loss: 0.6940 - val_categorical_accuracy: 0.5000 - val_recall_2: 0.5000 - val_precisio

In [14]:

#!pip install neptune-client
import os
import neptune
NEPTUNE_API_TOKEN="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiNjBlODllY2ItMDcyZC00OGNmLThjZWYtZjM2MjE4Y2M5ZDM0In0="
os.environ['NEPTUNE_PROJECT']="icassp/ECG"
#os.environ['NEPTUNE_NOTEBOOK_ID']="4897a223-8a9f-4f00-8cec-22eda690858d"
os.environ['NEPTUNE_NOTEBOOK_PATH']='ECG-simple (balanced).ipynb'

neptune.init('icassp/ECG',NEPTUNE_API_TOKEN)
exp = neptune.create_experiment(description='ECG-simple (balanced)',params=PARAMS,upload_source_files=['ECG-simple (balanced).ipynb'],tags=PARAMS['tags'],upload_stdout=True)

model.save(f'{model_dir}{str(exp)}.h5')
neptune.log_artifact(f'{model_dir}{str(exp)}.h5')

NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/icassp/ECG/e/ECG-26


In [15]:
no=2
for i in range(len(history.history['loss'])):
    neptune.log_metric('loss',history.history['loss'][i])
    neptune.log_metric('val loss',history.history['val_loss'][i])
    neptune.log_metric('acc',history.history['categorical_accuracy'][i])
    neptune.log_metric('val_acc',history.history['val_categorical_accuracy'][i])
    neptune.log_metric('precision',history.history['precision_%d'%no][i])
    neptune.log_metric('val_precision',history.history['val_precision_%d'%no][i])
    neptune.log_metric('recall',history.history['recall_%d'%no][i])
    neptune.log_metric('val_recall',history.history['val_recall_%d'%no][i])

In [ ]:
for i in range(len(history.history['loss'])):
    neptune.log_metric('loss',history.history['loss'][i])
    neptune.log_metric('val loss',history.history['val_loss'][i])
    neptune.log_metric('acc',history.history['categorical_accuracy'][i])
    neptune.log_metric('val_acc',history.history['val_categorical_accuracy'][i])
    neptune.log_metric('precision',history.history['precision'][i])
    neptune.log_metric('val_precision',history.history['val_precision'][i])
    neptune.log_metric('recall',history.history['recall'][i])
    neptune.log_metric('val_recall',history.history['val_recall'][i])

In [16]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        data_dir+'ecg_other_test/',  # this is the target directory
        target_size=(k1, k2),  # all images will be resized to 150x150
        batch_size=len(os.listdir(data_dir+'ecg_other_test/1'))+len(os.listdir(data_dir+'ecg_other_test/0')),
        class_mode='categorical')

X,y = test_generator.next()
prediction = model.predict(X)

import numpy as np

y_test=np.argmax(y,axis=1)
y_pred=prediction
from neptunecontrib.monitoring.metrics import *

threshold = 0.5

log_confusion_matrix(y_test, y_pred[:, 1] > threshold)
log_classification_report(y_test, y_pred[:, 1] > threshold)
log_class_metrics(y_test, y_pred[:, 1] > threshold)
log_class_metrics_by_threshold(y_test, y_pred[:, 1])
log_roc_auc(y_test, y_pred)
log_precision_recall_auc(y_test, y_pred)
log_brier_loss(y_test, y_pred[:, 1])
log_log_loss(y_test, y_pred)
log_ks_statistic(y_test, y_pred)
log_cumulative_gain(y_test, y_pred)
log_lift_curve(y_test, y_pred)
log_prediction_distribution(y_test, y_pred[:, 1])
neptune.stop()

Found 256 images belonging to 2 classes.
